In [1]:
# added path
import sys
import os

sys.path.append(os.path.abspath('..'))

In [2]:
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
from gpg.preprocess import Preprocess

In [4]:
# load ply
pcd = o3d.io.read_point_cloud('../data/desk_angledview.ply')

In [5]:
o3d.visualization.draw_geometries([pcd])

In [6]:
# preprocess
preprocess = Preprocess()
preprocessed_pcd, removed_indices = preprocess(pcd)

In [7]:
o3d.visualization.draw_geometries([preprocessed_pcd])

In [8]:
# vertex normal estimation
preprocessed_pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30)
)

In [9]:
# To display normals, type n key. Normals gets shorten by type - key
o3d.visualization.draw_geometries([preprocessed_pcd])

In [10]:
print('Print a normal vector of the 0th point')
print(preprocessed_pcd.normals[0])

Print a normal vector of the 0th point
[-0.08708295  0.57307426  0.81486345]


In [11]:
# convert normals to numpy
import numpy as np
print('Print the normal vectors of the first 10 points')
print(np.asarray(preprocessed_pcd.normals)[:10, :])

Print the normal vectors of the first 10 points
[[-0.08708295  0.57307426  0.81486345]
 [-0.20870645  0.48252378  0.85065411]
 [ 0.01088229  0.38999275  0.92075362]
 [ 0.01076836  0.39294729  0.91949794]
 [ 0.00492892  0.39180641  0.92003448]
 [-0.00597328  0.3983213   0.91722651]
 [ 0.01117813  0.3957075   0.91830857]
 [ 0.09806167  0.42949112  0.89773119]
 [ 0.14268844  0.42256279  0.89503112]
 [ 0.17255408  0.41358418  0.89396489]]


In [12]:
# segment plane
plane_model, inliers = preprocessed_pcd.segment_plane(distance_threshold=0.005, ransac_n=10, num_iterations=50)

In [13]:
plane_cloud = preprocessed_pcd.select_by_index(inliers)
plane_cloud.paint_uniform_color([1.0, 0.0, 0.0])
outlier_cloud = preprocessed_pcd.select_by_index(inliers, invert=True)
o3d.visualization.draw_geometries([plane_cloud, outlier_cloud])

In [27]:
# segmentation (bdscan)
import numpy as np
import matplotlib.pyplot as plt

labels = np.array(outlier_cloud.cluster_dbscan(eps=0.01, min_points=30))
max_label = labels.max()
colors = plt.get_cmap('tab20')(labels/max(max_label, 1))
colors[labels < 0] = 0
outlier_cloud.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([outlier_cloud])

In [45]:
c = 0
for i in labels:
    if i == 3:
        c += 1

In [46]:
c

1939